In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import zipfile

z= zipfile.ZipFile('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
z.extractall()

In [ ]:
train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')

In [ ]:
cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.describe()

In [ ]:
#we dont have to use id here
train.drop(['id'],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
import re

In [ ]:
train["comment_text"]=train["comment_text"].apply(lambda x: x.lower())

In [ ]:
train["comment_text"]=train["comment_text"].apply(lambda x: re.sub('\w*\d\w*','', x))

In [ ]:
test.shape

In [ ]:
import string
train["comment_text"]=train["comment_text"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

In [ ]:
def clean_text(text):
    text = re.sub(r"\n", " ", text)
    return text
train["comment_text"] = train["comment_text"].apply(clean_text)
train["comment_text"]

In [ ]:
train

In [ ]:
train.to_csv('train.csv',index=False)

In [ ]:
train.head()

In [ ]:
X = train.comment_text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect

In [ ]:
type(X)

In [ ]:
test.shape

In [ ]:
test_df = test.comment_text

In [ ]:
X_train = vect.fit_transform(X.astype('U'))
X_test = vect.transform(test_df.astype('U'))

In [ ]:
X_test

In [ ]:
X_train

In [ ]:
import sklearn

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=24.0)

In [ ]:
submission_binary1 = pd.read_csv('./sample_submission.csv')
submission_binary1.shape

In [ ]:
from sklearn.metrics import classification_report
for label in cols_target:
    print('... Processing {}'.format(label))
    y = train[label]
    # train the model using X_dtm & y
    logreg.fit(X_train, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_train)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    print(classification_report(y,y_pred_X))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(X_test)[:,1]
    print(test_y_prob.shape)
    submission_binary1[label] = test_y_prob

In [ ]:
train 

In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)

In [ ]:
train

In [ ]:
max_features = 20000
maxlen = 200

In [ ]:
list_sentences_train = train["comment_text"].values
list_sentences_test = test["comment_text"].values

In [ ]:
y = train[label_cols].values

In [ ]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)


In [ ]:
output_bias = None

In [ ]:

def get_model(output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid",bias_initializer=output_bias)(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


model = get_model()
print(model.summary())
batch_size = 32
epochs = 2

In [ ]:


file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)


callbacks_list = [checkpoint, early] #early
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

model.load_weights(file_path)

y_test = model.predict(X_te)



sample_submission = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip")

sample_submission[label_cols] = y_test



sample_submission.to_csv("baseline.csv", index=False)